In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from liamlib import dotdict
from tensorboardX import SummaryWriter
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
RUN_NAME = 'run'
writer = SummaryWriter(log_dir=f'./tboard/{RUN_NAME}')


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch, writer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            prog = 100. * batch_idx / len(train_loader)
            l = loss.item()
            batches = (epoch - 1) * len(train_loader.dataset) + batch_idx * len(data)
            writer.add_scalar('data/train_loss', l, batches)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), prog, l))

def test(args, model, device, test_loader, writer, batches):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), acc))
    writer.add_scalar('data/test_loss', test_loss, batches)
    writer.add_scalar('data/test_acc', acc, batches)

In [ ]:
args = dotdict.DotDict()

args.seed = 1
args.batch_size = 64
args.test_batch_size = 100
args.lr = 0.01
args.momentum = 0.5
args.epochs = 10
args.log_interval = 100

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, writer)
    test(args, model, device, test_loader, writer, epoch * len(train_loader))

if (args.save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438318
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.367301
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.327018
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.224800
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.336904
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.063205
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.116151
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.148756
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.116788

Test set: Average loss: 0.1014, Accuracy: 9662/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.144565
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.101065
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.058666
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.066316
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.121957
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.033579
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.033868
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.038529
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.028776
T